# workspace
- make it not add things together, treat each line an individual, if lines match, combine, make a merge helper
- create an output to things that are a reciept, perhaps a single output like "invalid"

In [4]:
from openai import OpenAI
import os
import pandas as pd

from dotenv import load_dotenv

_ = load_dotenv()

In [5]:
import base64
import re
import json
import time  # For tracking runtime
from dateutil.parser import parse as date_parse
from datetime import datetime


In [6]:
def encode_image(image_path: str) -> str:
    """
    Encode an image file to a base64 string.
    """
    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()
        image_base64 = base64.b64encode(image_bytes).decode("utf-8")
    return image_base64

In [7]:
def get_enhanced_receipt_items_gpt4o(image_path: str, current_date: str) -> str:
    """
    Sends a receipt image to the 4o-mini GPT model with instructions to return 
    items, quantities, a guessed expiration date, category, storage location, 
    and cost, in the format:
    
      {
        (cleaned_item1, quantity1, expiration1, category1, location1, cost1),
        (cleaned_item2, quantity2, expiration2, category2, location2, cost2),
        ...
      }
    
    We also provide a "current date" to GPT in the prompt so it can make
    a more educated guess about the expiration date.
    
    If the receipt is undecipherable or not a receipt, GPT should return "INVALID".
    """
    # Prompt that tells GPT exactly how to respond.
    # We provide the current_date in the instructions so it can reason about expiration.
    prompt = (
        f"Today's date is {current_date}. You are given a receipt image. Please parse each item on the receipt. "
        "For each item, do the following:\n"
        "1. Guess the cleaned-up item name (if uncertain, do your best or put N/A).\n"
        "2. Provide the quantity (if unknown, put N/A).\n"
        "3. Guess a reasonable expiration date for this food item based on today's date (if unsure, put N/A).\n"
        "4. Classify it from {F for Fruit, V for Vegetable, G for Grain, D for Dairy, M for Meat, S for Snack, O for Other} (if unsure, put N/A).\n"
        "5. Provide a storage location from {P for Pantry, F for Fridge, Z for Freezer} (if unsure, put N/A).\n"
        "6. Provide the cost as a float (e.g., 5.99). If unsure, put N/A.\n\n"
        "If the entire image is not a valid receipt or you cannot parse any items, return 'INVALID'.\n\n"
        "The final response format must be strictly:\n"
        "{(item, quantity, expiration, category, location, cost), ...}\n"
    )
    
    base64_image = encode_image(image_path)
    
    # Initialize the OpenAI client
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    # Create the chat completion request
    chat = client.chat.completions.create(
        model="gpt-4o-mini",
        max_tokens=1000,  # reducing can speed up runtime/cost
        temperature=0.0,  # lower, less creative (note, 0.0 makes better expiration dates)
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
    )
    
    # Extract the raw text output from GPT
    gpt_output = chat.choices[0].message.content
    return gpt_output

In [8]:
def parse_enhanced_gpt_output(gpt_output: str) -> list:
    """
    Given the GPT output in the format:
      {
        (cleaned_item, quantity, expiration, category, location, cost),
        ...
      }
    or "INVALID"
    
    - If it's "INVALID" or if no valid items were parsed, return an empty list.
    - Otherwise, parse each tuple and return a list of dictionaries, enforcing:
      {
        "item": string,
        "quantity": int,
        "expiration": ISO 8601 (or "N/A"),
        "category": one of [F, V, G, M, S, O],
        "location": one of [P, F, Z],
        "cost": float (or "N/A" if unparseable)
      }
    """
    # Check if the entire text is just "INVALID" (case-insensitive match)
    if gpt_output.strip().upper() == "INVALID":
        return []
    
    # Regex to capture 6 fields within parentheses:
    # (cleaned_item, quantity, expiration, category, location, cost)
    pattern = r'\(\s*([^,]+)\s*,\s*([^,]+)\s*,\s*([^,]+)\s*,\s*([^,]+)\s*,\s*([^,]+)\s*,\s*([^,]+)\s*\)'
    matches = re.findall(pattern, gpt_output)
    
    if not matches:
        return []
    
    valid_categories = {"F", "V", "G", "M", "S", "O"}
    valid_locations = {"P", "F", "Z"}
    
    items_list = []
    
    for match in matches:
        # match is a tuple of 6 strings
        cleaned_item, quantity, expiration, category, location, cost = match
        
        # Strip extra quotes/spaces
        cleaned_item = cleaned_item.strip("\"' ")
        quantity_str = quantity.strip("\"' ")
        expiration_str = expiration.strip("\"' ")
        category_str = category.strip("\"' ")
        location_str = location.strip("\"' ")
        cost_str = cost.strip("\"' ")
        
        # -------------------
        # 1) ITEM (string)
        item_val = cleaned_item
        
        # -------------------
        # 2) QUANTITY (int or "N/A")
        try:
            quantity_val = int(quantity_str)
        except ValueError:
            quantity_val = "N/A"  # fallback
        
        # -------------------
        # 3) EXPIRATION (ISO 8601 or "N/A")
        #    Attempt to parse user-provided date string:
        try:
            parsed_dt = date_parse(expiration_str)
            # Convert to ISO 8601 (YYYY-MM-DD) or full datetime .isoformat()
            expiration_val = parsed_dt.date().isoformat()
        except (ValueError, TypeError):
            expiration_val = "N/A"
        
        # -------------------
        # 4) CATEGORY
        #    Must be one of [F, V, G, M, S, O], else fallback to "O" or "N/A".
        category_val = category_str if category_str in valid_categories else "O"
        
        # -------------------
        # 5) LOCATION
        #    Must be one of [P, F, Z], else fallback to "P" or "N/A".
        location_val = location_str if location_str in valid_locations else "P"
        
        # -------------------
        # 6) COST (float or "N/A")
        try:
            cost_val = float(cost_str.replace("$", ""))  # remove $ if present
        except ValueError:
            cost_val = "N/A"
        
        items_list.append({
            "item": item_val,
            "quantity": quantity_val,
            "expiration": expiration_val,
            "category": category_val,
            "location": location_val,
            "cost": cost_val
        })
    
    return items_list

In [9]:
def write_json_to_file(data: list, filename: str = "receipt_output.json"):
    """
    Write the list of dictionaries to a JSON file.
    """
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

In [17]:
def main(image_path):
    # Track start time
    start_time = time.time()
    
    # Replace with the path to your receipt image
    #image_path = "comm4190_S25_Final_Project_GroupL/test_receipts/test1.png"
    
    # We supply today's date to help GPT figure out expiration
    # Example: "January 15, 2025" or "2025-01-15"
    current_date = datetime.now().strftime("%Y-%m-%d")  
    
    # 1. Get the raw enhanced text from GPT
    raw_gpt_output = get_enhanced_receipt_items_gpt4o(image_path, current_date)
    print("GPT Raw Enhanced Output:")
    print(raw_gpt_output)
    
    # 2. Parse the GPT output into a structured Python list
    items_json = parse_enhanced_gpt_output(raw_gpt_output)
    
    # 3. Check if it's invalid or empty
    if not items_json:
        print("\nNo valid items found or GPT returned 'INVALID'.")
    else:
        print("\nParsed JSON:")
        print(json.dumps(items_json, indent=2))

        # 4. Write the JSON to a file
        write_json_to_file(items_json, "receipt_output.json")
        print("\nThe parsed items have been written to 'receipt_output.json'.")

    # Track end time
    end_time = time.time()
    runtime_seconds = end_time - start_time
    print(f"\nScript runtime: {runtime_seconds:.2f} seconds")

In [19]:
main("test_receipts/test1.png")

GPT Raw Enhanced Output:
{ 
    ("St Louis Rib", 1, "2025-10-30", "M", "F", 21.63), 
    ("Organic Eggs", 1, "2025-06-30", "O", "F", 7.99), 
    ("Haddock", 1, "2025-10-30", "M", "F", 20.48), 
    ("Thai Jasmine Rice", 1, "2026-04-30", "G", "P", 19.99), 
    ("Silk Almond Milk", 1, "2025-09-30", "D", "F", 10.99), 
    ("NY Thin Cut Steak", 1, "2025-10-30", "M", "F", 39.12), 
    ("Brussels Sprouts", 1, "2025-06-30", "V", "F", 8.47), 
    ("Greek Yogurt", 1, "2025-08-30", "D", "F", 7.49), 
    ("Asparagus", 1, "2025-06-30", "V", "F", 7.49), 
    ("OMG3 Fish Oil", 1, "2026-04-30", "O", "P", 28.49), 
    ("Pumpkin Seed", 1, "2025-10-30", "O", "P", 3.60), 
    ("Organic Bananas", 1, "2025-05-15", "F", "P", 2.99), 
    ("3 Berry Mix", 1, "2025-06-30", "F", "F", 3.99), 
    ("Blueberries", 1, "2025-06-30", "F", "F", 3.99), 
    ("Avocados", 1, "2025-06-30", "F", "F", 4.99), 
    ("V8 Vegetable Juice", 1, "2026-04-30", "O", "P", 15.79), 
    ("V8 Vegetable Juice", 1, "2026-04-30", "O", "P", 1